In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import time

In [ ]:
def make_soup(url: str) -> BeautifulSoup:
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36"
    }

    # headers = headers, proxies = proxies ?
    source_code = requests.get(url, headers=headers, allow_redirects=True)
    if source_code.status_code == 503:
        print("blocked again...")
        assert False

    plain_text = source_code.text.encode("ascii", "replace")
    return BeautifulSoup(plain_text, "html.parser")

In [ ]:
URL = "https://www.andrewteale.me.uk/"

soup = make_soup("https://www.andrewteale.me.uk/leap/elections-index/")

all_links = []
for link in soup.find_all("a", {"href": re.compile(r"/results/")}):
    all_links.append(link["href"])

In [ ]:
for link in all_links:
    soup = make_soup(URL + link)

    x = soup.find(lambda tag: tag.name == "a" and "STV" in tag.text)
    if not x:
        continue

    v = x["href"].strip()
    zipfile = URL + v

    name = v.split("/")
    name = name[1] + "_" + name[2]

    r = requests.get(zipfile)
    with open("scraped/" + name, "wb") as f:
        f.write(r.content)

    time.sleep(1)